(introduction-to-simphony)=
# Introduction to simphony

Simphony is a Python package for that helps in defining and simulating photonic circuits, so having a basic understanding of Python will be helpful. In order to get started you will need to setup a python environment with simphony installed. If you are new to Python, we recommend using [Anaconda](https://www.anaconda.com/products/individual) to manage your Python environment. Once you have Anaconda installed, you can create a new environment and install the [``simphony``](https://pypi.org/project/simphony/) package by running the following commands in your terminal:

```bash
conda create -n simphony python=3.11
conda activate simphony
pip install simphony
```

Our goal with this tutorial is to give some of the background of the basics of simphony and [SAX](https://flaport.github.io/sax/), the underlying scattering parameter solver, in order to simulate a very simple photonic circuit. We'll go through the typical objects found in every circuit definition. We'll also show you how to simulate the circuit and obtain the results.

You can follow along with these tutorials in a Jupyter notebook by creating your own notebook or just downloading the notebook (.ipynb) for this tutorial in the top right of the page.

In [1]:
from jax import config
config.update("jax_enable_x64", True)

## Models

Models are the basic components in SAX, and they are used to represent an element in a photonic circuit. Models are simply functions that return s-parameters when called with the appropriate arguments. The s-parameter format in SAX is a dictionary of port-to-port scattering parameters. Not only is it easy to select which parameter you want simply using human-readable keys, but it also takes less memory for sparse matrices (those where most port relationships are 0).

Simphony has a number of built-in models coming from multiple included model libraries, but you can also create your own custom models by writing a function that implements the rules for a model. Here we make a simple custom model with two ports and one parameter:

In [2]:
import numpy as np
import sax
from jax.typing import ArrayLike

def custom_model(param: float = 0.5) -> sax.SDict:
    """This model will have one parameter, param, which defaults to 0.5."""
    # a simple wavelength independent s-matrix
    sdict = sax.reciprocal({
        ("in0", "out0"): -1j * np.sqrt(param),
    })
    return sdict
    
custom_model(param=0.25)

{('in0', 'out0'): -0.5j, ('out0', 'in0'): -0.5j}

```{important}
Model function parameters are required to be keyword-only. 

* In the backend, SAX inspects the model signature and passes in only the requested variables to the model function. 
* The global simulation parameters that are passed in must be named the same across all pertinent models.
* In order to determine the names of the ports in a model while building the netlist, SAX evaluates the model once without arguments. Hence, sensible defaults that run without raising any errors are required.
```

## Using Pre-built Models

```{eval-rst}
Simphony includes some pre-built models that can be used to build photonic circuits :py:mod:`simphony.libraries`:
``` 

One library we use comes from the [SiEPIC PDK](https://github.com/SiEPIC) (developed at the University of British Columbia). This library contains the s-parameters for a number of photonic components that were fabricated and measured for a number of different parameters.

For example lets instantiate two different models for a waveguide of different lengths:

In [3]:
from simphony.libraries import siepic

# waveguide of 2.5 mm length
wg1 = siepic.waveguide(length=2500, height=220)
# waveguide of 7.5 mm length
wg2 = siepic.waveguide(length=7500, height=210)

```{eval-rst}
These are both :py:meth:`~simphony.libraries.siepic.waveguide` models. The SiEPIC components are parameterizable, so we can pass different parameters when instantiating them. In this case ``wg1`` will be a thicker waveguide than ``wg2``. The two models will have different s-parameters since their thicknesses are different.
```

```{note}
The convention in simphony is to use microns for units of length.
```

## Creating a Circuit

Ports are represented simply by string names in SAX, and a netlist (or a circuit) is simply a dictionary of models, their connections, and the subsequently exposed ports. Netlists can also be used as models in other netlists; the ports of that composite model are the ports exposed by your netlist.

The simplest way to create a circuit and connect ports is as follows:

In [5]:
netlist = {
    "instances": {
        "wg1": "waveguide",
        "wg2": "waveguide",
    },
    "connections": {
        "wg1,o1": "wg2,o0",
    },
    "ports": {
        "in": "wg1,o0",
        "out": "wg2,o1",
    }
}

You then create a circuit object and define which methods belong to your models. This makes it very easy to swap other models in and out during different simulations.

In [6]:
circuit, info = sax.circuit(
    netlist=netlist,
    models={
        "waveguide": siepic.waveguide,
    }
)

By default, models in simphony have port names prefixed with "o" and increasing from 0, e.g. "o0", "o1", etc. Above, we specified "o1" of ``wg1`` must connect to "o0" of ``wg2``. We gave our overall circuit more useful port names, though; "in" and "out" are more descriptive than "o0" and "o1".

For each model you use, refer to its documentation to see how port names are assigned.

With this netlist defined, we now have a rudimentary circuit to run simulations on.

## Simulation

```{eval-rst}
:py:mod:`simphony.simulators` provides a collection of simulators that connect to an input and output pin on a
circuit, then perform a series of calculations to solve for the relationship between output light at each port of the circuit for given inputs of light.
```

Let's run a simple sweep simulation on the circuit we have created. We can pass parameters to the individual components within our circuit using the names we gave them in our netlist--that is, "wg1" and "wg2". We pass them dictionaries of keyword arguments that will be used to instantiate their given models, so the key names must match those in the parameter list of the model. Always check the API of the models you want to use to see what parameters they take.

In [7]:
from simphony.classical import ClassicalSim

# Create a simulation and add a laser and detector
sim = ClassicalSim(circuit, wl=1.55, wg1={"length": 2500.0, "loss": 3.0}, wg2={"length": 7500.0, "loss": 3.0})
laser = sim.add_laser(ports=["in"], power=1.0)
detector = sim.add_detector(ports=["out"])

# Run the simulation
result = sim.run()

# Since the total wg length is 1 cm and the loss is 3 dB/cm, the power should be approximately 50%.
print(f"Power transmission: {abs(result.sdict['out'])**2}")

Power transmission: [0.50118723]


We instantiated up our simulator with our circuit, adding a laser input to the "input" port of ``wg1`` and placing a detector on
``wg2``. Our sweep simulation passed input light on a range of wavelengths from 1.5 microns to
1.6 microns, and now ``result`` contains what frequencies came out of our circuit. We can use these results however we like.

In order to view the results, we can use the ``matplotlib`` package to graph our output, but that will be demonstrated
in following tutorials. For this tutorial, we're done!